<a href="https://colab.research.google.com/github/CaseyAnnHorvath/desihigh/blob/main/two_body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/desihigh/')

In [3]:
import time
import astropy
import itertools
import matplotlib

import numpy             as np
import pylab             as pl
import matplotlib.pyplot as plt
import astropy.units     as u

from   astropy.cosmology import FlatLambdaCDM
from   IPython.display   import YouTubeVideo
from   tools.flops       import flops

In [4]:
from IPython.display import clear_output
from time import sleep

In [5]:
%matplotlib inline

plt.style.use('dark_background')

In [6]:
def init_dof(npt=1):
  #  Create a set of particles at random positions in a box, which will soon predict the distribution of dark matter 
  #  as we see above.
  xs     = np.random.uniform(0., 1., npt)
  ys     = np.random.uniform(0., 1., npt)
  zs     = np.random.uniform(0., 1., npt)

  pos    = np.vstack((xs,ys,zs)).T
  vel    = np.zeros_like(pos)

  return  pos, vel

In [7]:
npt = 2
pos, vel  = init_dof(npt=npt)
mass=np.zeros_like(pos)


In [8]:
i = 0
pos[i][0]=0
pos[i][1]=0
pos[i][2]=0
pos[0]

i = 1
pos[i][0]=1.5e12
pos[i][1]=0
pos[i][2]=0



# vel of particle 1
i=0

vel[i][0]=0
vel[i][1]=11000
vel[i][2]=0

i=1

vel[i][1]=0
vel[i][1]=-5500
vel[i][2]=0



mass[0]=1.5e30
mass[1]=6e30

In [9]:
def com(pos,mass):  #defining CoM function, 
  result=np.dot(mass*pos)/np.sum(mass)
  return result

In [10]:
#Rcom=com(pos,mass)

In [11]:
print, Rcom

NameError: ignored

In [12]:
pos_com=pos-Rcom
print, pos_com

NameError: ignored

In [13]:
#defining original values for the 2 bodies

m1=1.5e30
m2=6e30

v1=1.1e4
v2=-5.5e3

r1=0
r2=1.5e12

#distance between m1 and m2
r=r2-r1


In [14]:
def com_vel(vel,mass):
  result=np.sum(mass*vel,axis=0)/np.sum(mass)
  return result

#Simple 2body version

def findCoM(m1, r1, v1, m2, r2, v2):
  return (m1*r1 + m2*r2)/(m1+m2), (m1*v1 + m2*v2)/(m1+m2)

In [15]:
#dont need anymore
#print(findCoM(m1, r1, v1, m2, r2, v2)) #1.5e30, 0, 11000, 6e30, 1.5e12, -5500
#print, com_vel(vel,mass)

In [16]:
com_pos, com_vel = findCoM(m1, r1, v1, m2, r2, v2)
print(com_pos, com_vel)

1200000000000.0 -2200.0


In [17]:
#setting com reference frome to 0 vel

v1=v1-com_vel
v2=v2-com_vel

print(v1,v2)

13200.0 -3300.0


In [18]:
#Gravitational Constant
G=6.67e-11

In [19]:
#energy calculation


def energy(m1,v1,m2,v2,r):
  return .5*m1*(v1**2)+.5*m2*(v2**2)-((G*m1*m2)/r)
  

In [20]:
E=energy(m1,v1,m2,v2,r)
print(E)

-2.3684999999999995e+38


In [21]:
#angular momentum calculation
def ang_momentum(m1,m2,v1,v2,r1,r2):
  return m1*v1*r1+m2*v2*r2

In [22]:
L=ang_momentum(m1,m2,v1,v2,r1,r2)
print(L)

-2.9699999999999997e+46


In [23]:
#calc reduced mass
def reduced_mass(m1,m2):
  return (m1*m2)/(m1+m2)
  

In [24]:
u=reduced_mass(m1,m2)
print(u)

1.1999999999999998e+30


In [25]:
#calc total mass
def tot_mass(m1,m2):
  return m1+m2

In [26]:
M=tot_mass(m1,m2)
print(M)

7.5e+30


In [27]:
#calc eccentricity
def eccentricity(L,E,u,M):
  return (1+(2*(L**2)*E)/((u**3)*(G**2)*(M**2)))**(1/2)

In [28]:
e=eccentricity(L,E,u,M)
print(e)

0.183658170914542


In [29]:
#calc semi major axis
def sem_maj(M,u,E):
  return (G*M*u)/(2*E)
  

In [30]:
a=sem_maj(M,u,E)
print(a)

-1267257758074.731


In [31]:
#calculating distance for perhelion (maybe dont need)
def perihelion_rad(a,e):
  return a*(1-e)

r_peri=perihelion_rad(a,e)
print(r_peri)

-1034515516149.4628


In [32]:
#calculating distance for aphelion (maybe dont need)
def aphelion_rad(a,e):
  return a*(1+e)

r_ap=aphelion_rad(a,e)
print(r_ap)

-1499999999999.9993


In [33]:
#calc location of m1 and m2 (maybe dont need)


In [72]:
#calc theta-theta knot 

def theta(L,u,M,r,e,G):
  arg=((L**2)/(G*M*(u**2)*r*e))-(1.0/e)
  
  return np.arccos(np.around(arg,4))


In [73]:
theta=theta(L,u,M,r,e,G)
print(theta)

3.141592653589793


In [108]:
def rho_(L,u,M,r,e,G,theta,d_theta):
  return ((L**2)/(G*M*(u**2)*(1.0+e*np.cos(d_theta+theta))))

In [110]:
rho= rho_(L,u,M,r,e,G,theta,0)
print(rho)



1069403265914.8223


In [111]:
def rho_x(rho, theta,d_theta):
  return rho*np.cos(d_theta+theta)

In [112]:
rho_x= rho_x(rho, theta,0)
print(rho_x)

844555278651.5421


In [113]:
def rho_y(rho, theta,d_theta):
  return rho*np.sin(d_theta+theta)

In [114]:
rho_y= rho_y(rho, theta,0)
print(rho_y)

-656010462150.646


In [115]:
def r_1(u,m1,rho, theta):
  r1=(-u/m1)*rho
  r1_x=(-u/m1)*rho*np.cos(theta)
  r1_y=(-u/m1)*rho*np.sin(theta)
  return r1, r1_x, r1_y

In [116]:
r1, r1_x, r1_y=r_1(u,m1,rho,theta)
print(r1, r1_x, r1_y)

-855522612731.8578 -675644222921.2336 524808369720.5168


In [117]:
def r_2(u,m2,rho, theta):
  r2=(u/m1)*rho
  r2_x=(u/m1)*rho*np.cos(theta)
  r2_y=(u/m2)*rho*np.sin(theta)
  return r2, r2_x, r2_y

In [118]:
r2, r2_x, r2_y=r_2(u,m2,rho,theta)
print(r2, r2_x, r2_y)

855522612731.8578 675644222921.2336 -131202092430.1292


In [122]:
dt         = 365*24*3600
nsteps     = 500
rho1=rho_(L,u,M,r,e,G,theta,0)
timesteps  = np.linspace(0, (nsteps)*dt, nsteps, endpoint=False)
for index_in_timestep, time in enumerate(timesteps):
       dtheta = L/u/rho**2*dt
       theta += dtheta
       rho1=rho_(L,u,M,r,e,G,theta,0)
       r1, r1_x, r1_y=r_1(u,m1,rho1,theta)
       r2, r2_x, r2_y=r_2(u,m2,rho1,theta)
